In [1]:
from sklearn.linear_model import SGDClassifier
from gensim import corpora, models, similarities
import numpy as np
import shelve
import time
from scipy.sparse import csr_matrix
from sklearn import metrics
shelve_file = shelve.open('../data/sample')
X_train = shelve_file['X_train']
y_train = shelve_file['y_train']
X_test = shelve_file['X_test']



def func(documents):
    texts = [[word for word in document.split()] for document in documents]
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    corpus_lsi = lsi[corpus_tfidf]
    rows_new = []
    cols_new = []
    data_new = []
    line_count = 0
    for line in corpus_lsi:
        for elem in line:
            rows_new.append(line_count)
            cols_new.append(elem[0])
            data_new.append(elem[1])
        line_count += 1
    lsi_sparse_matrix = csr_matrix((data_new, (rows_new, cols_new)))
    return lsi_sparse_matrix

lsi_start = time.time()
texts = [[word for word in document.split()] for document in X_train]
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below = 5,no_above = 0.9)
dictionary.compactify()
corpus = [dictionary.doc2bow(text) for text in texts]

tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=7200)
corpus_lsi = lsi[corpus_tfidf]
print("Time consumption on lsi",time.time( ) - lsi_start)

trans_start = time.time()
row = []
rows = []
cols = []
data = []
line_count = 0
for line in corpus_lsi:
    for elem in line:
        rows.append(line_count)
        cols.append(elem[0])
        data.append(elem[1])
    line_count += 1
lsi_sparse_matrix = csr_matrix((data, (rows, cols)))
lsi_matrix = lsi_sparse_matrix.toarray()
print("Time consumption on transform",time.time( ) - trans_start)
print(lsi_matrix.shape)
'''

'''

Time consumption on lsi 9527.16878771782
Time consumption on transform 1542.5663554668427
(100562, 7200)


'\n\n'

In [2]:
#print(X_test.shape)

In [3]:

train_start = time.time()
X_train = lsi_sparse_matrix
clf = SGDClassifier(loss="modified_huber", penalty="l2", alpha=0.00002, n_jobs=-1)#分类器
clf.fit(X_train,y_train)
print('Time comsumption on train',time.time()-train_start)



prediction = clf.predict(X_train)
print(metrics.classification_report(y_train,prediction))#分类报告
print(metrics.confusion_matrix(y_train,prediction))#混淆矩阵


test_start = time.time()
#预测
X_test = func(X_test)

prediction = clf.predict(X_test)
print('Time comsumption on test',time.time()-test_start)
#保存结果
save_start = time.time()
f_out = open('ans.csv', 'w')
f_out.write("id,class"+"\n")
for i in range(X_test.shape[0]):
    f_out.write(str(i)+","+str(prediction[i])+'\n')
f_out.close()
print('Time comsumption on save',time.time()-save_start)

/home/amax/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Time comsumption on train 30.33351182937622
             precision    recall  f1-score   support

          1       0.71      0.67      0.69      5341
         10       0.80      0.76      0.78      4927
         11       0.81      0.81      0.81      3514
         12       0.79      0.79      0.79      5242
         13       0.82      0.85      0.83      7818
         14       0.81      0.87      0.84      6693
         15       0.92      0.94      0.93      7082
         16       0.79      0.61      0.69      2981
         17       0.80      0.83      0.82      3058
         18       0.89      0.93      0.91      6970
         19       0.72      0.75      0.74      5483
          2       0.87      0.86      0.86      2889
          3       0.94      0.92      0.93      8217
          4       0.89      0.90      0.90      3751
          5       0.90      0.87      0.89      2335
          6       0.95      0.94      0.95      6804
          7       0.83      0.74      0.79      3010
 

MemoryError: 